# Import
Importamos las librerias importantes

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

Bajamos el martin fierro

In [ ]:
!wget https://www.gutenberg.org/files/14765/14765-8.txt -O martin_fierro.txt

--2021-11-01 22:21:06--  https://www.gutenberg.org/files/14765/14765-8.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89471 (87K) [text/plain]
Saving to: ‘martin_fierro.txt’

martin_fierro.txt   100%[===================>]  87.37K   160KB/s    in 0.5s    

2021-11-01 22:21:07 (160 KB/s) - ‘martin_fierro.txt’ saved [89471/89471]





```
# This is formatted as code
```

Aca se genera el encoder y se arma el dataset

In [ ]:
# Leer el archivo y arma el vocabulario(todos los caracteres que aparecen)
text = open("martin_fierro.txt", 'rb').read().decode(encoding='latin-1')
# Limpia para que solo quede el texto del martin fierro
text = text[4945:70218]
vocab = sorted(set(text))

# Arma un lookup para traducir de string a un numero y viceversa
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab), mask_token=None)
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
# Funcion helper para traducir un conjunto de ids a texto
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

# Traduce el texto a ids
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text
dataset = sequences.map(split_input_target)


# Batch size
BATCH_SIZE = 64
# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

# Arma el dataset
dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))


Aca se arma el modelo

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

# Arma un modelo de tres capas

class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    # Transforma el id en un vector de tamaño fijo
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    # Red neuronal recurrente
    self.lstm = tf.keras.layers.LSTM(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    # Red neuronal normal
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.lstm.get_initial_state(x)
    x, final_state,carry_state = self.lstm(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, [final_state,carry_state]
    else:
      return x

Se instancia el modelo y se le agrega la funcion de perdida y la forma en la que se optimizara

In [ ]:
model = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

# Configura el modelo para ser entrenado
model.compile(optimizer='adam', loss=loss)

# Configuracion para guardar el modelo en un archivo
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

Aca se entrena

In [ ]:
# Aca se entrena el modelo
EPOCHS = 100
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


Epoch 1/100
10/10 [==============================] - 2s 169ms/step - loss: 0.2085
Epoch 2/100
10/10 [==============================] - 2s 156ms/step - loss: 0.1939
Epoch 3/100
10/10 [==============================] - 2s 153ms/step - loss: 0.1808
Epoch 4/100
10/10 [==============================] - 2s 154ms/step - loss: 0.1701
Epoch 5/100
10/10 [==============================] - 2s 154ms/step - loss: 0.1594
Epoch 6/100
10/10 [==============================] - 2s 156ms/step - loss: 0.1484
Epoch 7/100
10/10 [==============================] - 2s 151ms/step - loss: 0.1378
Epoch 8/100
10/10 [==============================] - 2s 153ms/step - loss: 0.1295
Epoch 9/100
10/10 [==============================] - 2s 153ms/step - loss: 0.1234
Epoch 10/100
10/10 [==============================] - 2s 155ms/step - loss: 0.1181
Epoch 11/100
10/10 [==============================] - 2s 155ms/step - loss: 0.1118
Epoch 12/100
10/10 [==============================] - 2s 155ms/step - loss: 0.1053
Epoch 13/100


Se genera un modelo que permita generar texto a partir de otro modelo

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

Se genera texto a partir del modelo previamente entrenado

In [ ]:
# instancia lo necesario para generar el texto
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)
states = None
next_char = tf.constant(['hermanos:'])
result = [next_char]
# Genera 1000 predicciones
for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

hermanos:
pues a voca me la imerra
hagará el pobre solidoía;
pero a hacer más cantones
con las prendas a un garcullo,
se hace una cospellario
y ver nuega al cacillones
y ansí de entonde ista,
gané te9gamos, la gaucha;
viendo esperen pa nacirá
hasta que por su señañaba
que ellos la palge en la meno;
ya le acopo la escuché.

293
Amigo, en a la frontera
hagá el gaucho como dargar,
aunque mescos en muerto,
lo mesmito que en pastula
Y voz empezado el verzón
como a quererme comersa;
mas viene e cuampa lunita
de camitos cuen porrón.

392
Ansí estros al campo algo
ya escucho anda en mi cama,
Y tuvié una remalón
más bien apuantando yo,
no pedía comperiza el crieno,
y así andaba divía en un diablo
sin dejalle raliga junto,
y aunque Dios tanda galones
con que se trabaja cruza.

346
En medio ese cases
cantandome media mi cuido
y pa nigar cualta velle,
que dende que otra puerde
la gaucha contro aguanda.

88
Tal vez el cristian salbie
tuanto un buale en la penza!
 

_________________________________